In [13]:
%run train.py

AttributeError: 'Output' object has no attribute 'capture'

In [18]:
    
s = Segmentation(
    'tls_park', 
    training_repo='/data1/sat/dataset_tls_park_444/train.txt',
    testing_repo='/data1/sat/dataset_tls_park_444/test.txt',
    host='10.10.77.61',
    port=8500,
    #port=8400,
    img_height=480,
    img_width=480,
    model_repo='/data1/sat/xolive/models/model_tls_park_debug',
    nclasses=2,
    #template='deeplab_vgg16',
    iterations=15000,
    test_interval=500,
    snapshot_interval=1000,
    batch_size=5,
    test_batch_size=1,
    noise_prob=0.001,
    class_weights=[0.1, 1.0],
    model_postfix='test_park_xo3',
    weights='/data1/segmentation/vgg16_init_deeplab.caffemodel',
    gpuid=2
)#.run()
s

In [19]:
c = Classification(
    'xo_bbc_5_classes', 
        training_repo='/data1/xolive/bbc_sounds/sounds_dataset_1.0/train/',
        testing_repo='/data1/xolive/bbc_sounds/sounds_dataset_1.0/test/',
                   host='10.10.77.61',
        port=8500,
        model_repo='/data1/bbc_sounds/bbc_5_classes',
        layers='["1CR32", "1CR64", "1CR128", "1024"]',
        template='convnet',
        gpuid=1,
                   img_width=257,
                   img_height=257
        )#.run()
c
# TODO unchanged data

In [20]:
c = Classification('kaggle', 
                    training_repo='/data1/kaggle/dogs_cats/train/',
                   tsplit=.2,
                    host='10.10.77.61',
                    port=8500,
                    model_repo='/data1/xolive/models/kaggle',
                    template='convnet',
                    gpuid=1,
                   layers=['1CR32','1CR64','1CR128','1024'],
                   img_width=150,
                   img_height=150,
                   mirror=True,
                   rotate=True,
        )#.run()
c
# TODO unchanged data

In [60]:
%run train.py

d = Detection(
    "eoliennes",
    training_repo= "/data1/sat/dataset_windturbine_europe_16_test/train.txt",
    testing_repo= "/data1/sat/dataset_windturbine_europe_16_test/test.txt",
    host='10.10.77.61',
    port=8500,
    model_repo='/data1/xolive/models/detection',
    gpuid=0,
    img_width=300,
    img_height=300,
    snapshot_interval=1000,
    iterations=5000,
    template="ssd_300",
    mirror=True,
    rotate=True, # no default
    finetune=True,
    weights="/data1/detection/VGG_ILSVRC_16_layers_fc_reduced.caffemodel",
    batch_size=24,
    test_batch_size=4,
    nclasses=2,
)
d